In [1]:
import gurobipy as gp
from gurobipy import GRB
import plotly.express as px
import pandas

In [2]:
df_costo_cliente_planta = pandas.read_excel("./data.xlsx", sheet_name="COSTO CLIENTE-PLANTA", index_col=[0, 1])
df_demanda = pandas.read_excel("./data.xlsx", sheet_name="DEMANDAS CLIENTES", index_col=0)
df_plantas = pandas.read_excel("./data.xlsx", sheet_name="COSTO Y CAPACIDAD PLANTAS", index_col=0)

In [3]:
PLANTAS = df_plantas.index.to_list()
CLIENTES = df_demanda.index.to_list()

In [4]:
def model_plantas_clientes(min_capacidad: float, tiempo_ejecucion_sec: int) -> float:

    model = gp.Model()

    ## Variables
    x = model.addVars(PLANTAS, vtype=GRB.BINARY, name="x")
    y = model.addVars([(planta, cliente) for planta in PLANTAS for cliente in CLIENTES], vtype=GRB.BINARY, name="y")

    ## Función Objetivo
    costo_abrir_plantas = gp.quicksum(x[planta] * df_plantas.loc[planta, "Costo de abrir"] for planta in PLANTAS)
    costo_cliente_planta = gp.quicksum(y[(planta, cliente)] * df_costo_cliente_planta.loc[(planta, cliente), "Costo de asignar un cliente a un planta"] for planta in PLANTAS for cliente in CLIENTES)
    model.setObjective(costo_abrir_plantas + costo_cliente_planta, GRB.MINIMIZE)

    ## Restricciones
    for planta in PLANTAS:
        for cliente in CLIENTES:
            model.addConstr(y[(planta, cliente)] <= x[planta], name=f"ct1_{planta}_{cliente}")

    for planta in PLANTAS:
        model.addConstr(gp.quicksum(y[(planta, cliente)] * df_demanda.loc[cliente, "Demanda"] for cliente in CLIENTES) <= df_plantas.loc[planta, "Capacidad máxima"], name=f"ct2_{planta}")

    for cliente in CLIENTES:
        model.addConstr(gp.quicksum(y[(planta, cliente)] for planta in PLANTAS) == 1, name=f"ct3_{cliente}")

    for planta in PLANTAS:
        model.addConstr(gp.quicksum(y[(planta, cliente)] for cliente in CLIENTES) >= 3 * x[planta], name=f"ct4_{planta}")

    for planta in PLANTAS:
        model.addConstr(
            gp.quicksum(y[(planta, cliente)] * df_demanda.loc[cliente, "Demanda"] for cliente in CLIENTES) >= min_capacidad * df_plantas.loc[planta, "Capacidad máxima"] * x[planta],
            name=f"ct5_{planta}",
        )

    ## Resolver Modelo
    model.setParam("TimeLimit", tiempo_ejecucion_sec)
    model.optimize()

    return round(model.ObjVal, 2)

In [5]:
CAPACS = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
FOS = []
for capac in CAPACS:
    fo = model_plantas_clientes(capac, 200)
    FOS.append(fo)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-28
Set parameter TimeLimit to value 200
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1110 rows, 1020 columns and 6040 nonzeros
Model fingerprint: 0x777e123b
Variable types: 0 continuous, 1020 integer (1020 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 226 rows and 74 columns
Presolve time: 0.13s
Presolved: 884 rows, 946 columns, 6131 nonzeros
Variable types: 0 continuous, 946 integer (946 binary)

Root relaxation: objective 6.319329e+03, 1108 iterations, 0.07 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds  

In [ ]:
fig = px.line(x=CAPACS, y=FOS, markers=True)

fig.update_traces(
    marker=dict(color="red", size=7, symbol="x"),
    line=dict(color="blue", width=3),
)

fig.update_layout(
    coloraxis_showscale=False,
    height=400,
    width=600,
    xaxis={"title": "Capacidades"},
    yaxis={"title": "Función Objetivo"},
    title="Capacidad VS Función Objetivo",
    template="ggplot2",
)

fig.show()
